# Setup

In [12]:
!pip install pkbar

In [ ]:
from google.colab import drive
from google.colab import files
import sys
import time

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/PSGD Paper/'
results_dir = base_dir + 'RNN/results_xor/'
logs_dir = base_dir + 'log'
sys.path.append(base_dir)
import preconditioned_stochastic_gradient_descent as psgd 


Mounted at /content/gdrive/


In [13]:
import matplotlib.pyplot as plt
import torch
from torch.autograd import grad
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import time
import math

import pkbar
from tqdm import tqdm
from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px
from torchsummary import summary
import torch.nn as nn
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.get_device_name(0)


'Tesla T4'

In [27]:
model = nn.Sequential(
      nn.Embedding(20000, 256),
      nn.LSTM(256, 128),
      nn.Linear(128, 1)
)
[p.size() for p in model.parameters()]

[torch.Size([20000, 256]),
 torch.Size([512, 256]),
 torch.Size([512, 128]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([1, 128]),
 torch.Size([1])]

# Functions

In [ ]:
def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):

    fig = go.Figure()
    i = 0
    if(xaxis != None):
        for opt in opts:
            fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1
    else:
        for opt in opts:
            fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1

    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
    fig.show()
    fig.write_html(results_dir + title + ".html")

def plot_acc_metrics(xaxis,yaxis,title, x_label,y_label):
 
    fig = go.Figure()
    i = 0
    if(xaxis != None):
        for opt in opts:
            fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1
    else:
        for opt in opts:
            fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
            i = i + 1

    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis=dict(range=[0.97, 1]))
    fig.show()
    fig.write_html(results_dir + title + ".html")


def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    if r > 3/4:
        lambd = lambd*omega
    elif r < 1/4:
        lambd = lambd / omega
    return lambd
    

In [ ]:
np.random.seed(0)

# Parameter Settings
BATCH_SIZE = 100
test_BATCH_SIZE = 100
EPOCHS = 10
GAP = 100

# Data Download

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing import sequence

from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np

from sklearn.datasets import fetch_20newsgroups
max_features = 20000

maxlen = 80
batch_size = 32


def load_20ng_dataset_bow():
   
    k=['alt.atheism','rec.autos']
    newsgroups_train = fetch_20newsgroups(subset='train',categories=k)
    newsgroups_train.target_names
   
    newsgroups_test = fetch_20newsgroups(subset='test',categories=k)
    newsgroups_test.target_names

   

    vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.95)
    train_data = vectorizer.fit_transform(newsgroups_train.data)
    test_data = vectorizer.transform(newsgroups_test.data)
    train_data = train_data.todense()
    test_data = test_data.todense()
    train_labels = newsgroups_train.target
    test_labels = newsgroups_test.target

    return train_data, train_labels, test_data, test_labels

   


np.random.seed(1)
n_train = 715
   
X_train, y_train, X_test, y_test = load_20ng_dataset_bow()
X_train = X_train[:n_train, :]
y_train = y_train[:n_train]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train),torch.Tensor(y_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers=4)

test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test),torch.Tensor(y_test))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [30]:
n_batches = len(train_loader)
n_test_batches = len(test_loader)

# Model

In [ ]:
# generate a random orthogonal matrix for recurrent matrix initialization 
dim_in = 256
dim_hidden = 128
batch_size = 32
# initialize the RNN weights

def initialize_weights():
    
    Wi_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden+1, dim_hidden])
    Wf_np = np.concatenate([np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden, dim_hidden]),
                                    np.ones([1, dim_hidden])], 0)
    Wo_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden+1, dim_hidden])
    Wc_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_in+2*dim_hidden+1, dim_hidden])
    # matrix for the output layer (not for LSTM cell)
    W2_np = np.random.normal(loc=0.0, scale=0.1, size=[dim_hidden+1, dim_out])

    W1 = torch.nn.init.xavier_uniform_((0.1*torch.randn(vocab_size, input_dim)).clone().detach().requires_grad_(True)).to(device)
    Wi = torch.tensor(Wi_np, dtype=torch.float, requires_grad=True).to(device)
    Wf = torch.tensor(Wf_np, dtype=torch.float, requires_grad=True).to(device)
    Wo = torch.tensor(Wo_np, dtype=torch.float, requires_grad=True).to(device)
    Wc = torch.tensor(Wc_np, dtype=torch.float, requires_grad=True).to(device)
    W2 = torch.tensor(W2_np, dtype=torch.float, requires_grad=True).to(device)
    Ws = [Wi, Wf, Wo, Wc, W2]
    return Ws

def get_rand_orth( dim ):
    temp = np.random.normal(size=[dim, dim])
    q, _ = np.linalg.qr(temp)
    return q

ones = torch.ones(batch_size, 1).to(device)


def model(x):
    Wi, Wf, Wo, Wc, W2 = Ws
    
    def lstm_cell(i, o, state):
        # i: input
        # o: output
        # state: cell state
        input_gate = torch.sigmoid(torch.matmul(torch.cat([i, o, state, ones], 1), Wi))
        forget_gate = torch.sigmoid(torch.matmul(torch.cat([i, o, state, ones], 1), Wf))
        update = torch.matmul(torch.cat([i, o, state, ones], 1), Wc)
        state = forget_gate*state + input_gate*torch.tanh(update)
        
        output_gate = torch.sigmoid(torch.matmul(torch.cat([i, o, state, ones], 1), Wo))
        o = output_gate*torch.tanh(state)
        return o, state
    
    state = torch.zeros([batch_size, dim_hidden]).to(device)
    out = torch.zeros([batch_size, dim_hidden]).to(device)

    for xt in x.transpose(1,0):
        out, state = lstm_cell(xt, out, state)
    y = torch.matmul(torch.cat((out, ones), 1), W2)
    return y

In [ ]:
torch.manual_seed(1)
np.random.seed(0)
initialize_weights()

[tensor([[ 1.7641e-01,  4.0016e-02,  9.7874e-02,  2.2409e-01,  1.8676e-01,
          -9.7728e-02,  9.5009e-02, -1.5136e-02, -1.0322e-02,  4.1060e-02,
           1.4404e-02,  1.4543e-01,  7.6104e-02,  1.2168e-02,  4.4386e-02,
           3.3367e-02,  1.4941e-01, -2.0516e-02,  3.1307e-02, -8.5410e-02],
         [-2.5530e-01,  6.5362e-02,  8.6444e-02, -7.4216e-02,  2.2698e-01,
          -1.4544e-01,  4.5759e-03, -1.8718e-02,  1.5328e-01,  1.4694e-01,
           1.5495e-02,  3.7816e-02, -8.8779e-02, -1.9808e-01, -3.4791e-02,
           1.5635e-02,  1.2303e-01,  1.2024e-01, -3.8733e-02, -3.0230e-02],
         [-1.0486e-01, -1.4200e-01, -1.7063e-01,  1.9508e-01, -5.0965e-02,
          -4.3807e-02, -1.2528e-01,  7.7749e-02, -1.6139e-01, -2.1274e-02,
          -8.9547e-02,  3.8690e-02, -5.1081e-02, -1.1806e-01, -2.8182e-03,
           4.2833e-02,  6.6517e-03,  3.0247e-02, -6.3432e-02, -3.6274e-02],
         [-6.7246e-02, -3.5955e-02, -8.1315e-02, -1.7263e-01,  1.7743e-02,
          -4.0178e-02,

# Loss Function

In [ ]:
def train_loss(data, target):
    out = model(data)
    loss = F.mse_loss(out, target)
    return loss

def test_loss():
    loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            out = model(data)
            loss += F.mse_loss(out, target)
            
    return loss.item()/n_test_batches


def save_start_condition(trainlosslist, testlosslist, timelist):
    trainloss = 0.0

    for (data, target) in tqdm(train_loader, ncols = 80):
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        trainloss += loss

    timelist.append(0)

    testloss = test_loss()

    trainlosslist.append(trainloss.item()/n_batches)
    testlosslist.append(testloss)
    print('Epoch: {}; train loss: {}; test loss: {},  time: {}'.format(0, trainlosslist[-1], testlosslist[-1], np.sum(timelist)))

# SGD

In [ ]:
torch.manual_seed(1)
np.random.seed(0)
Ws = initialize_weights()
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.numel() for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
    save_start_condition(TrainLoss, TestLoss, times)
for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for (data, target) in train_loader:
        
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        trainloss += loss
        grads = grad(loss, Ws)  
        

        with torch.no_grad():
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for (W,pG) in zip(Ws, grads):
                W -= step_adjust*step_size*pG
            kbar.update(n, values=[("loss", loss.item())])    
            n = n + 1
            
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    
    
#     step_size = 0.01**(1/9)*step_size
    testloss = test_loss()
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
#     print('Epoch: {}; train loss: {}; test loss: {}, train_acc: {}, test_acc:{}, time: {}'\
#      .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'SGD.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|████████████████████████████████████████| 450/450 [00:03<00:00, 124.56it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 10s 22ms/step - loss: 0.1525 - val_loss: 0.1463
Epoch: 2/10
450/450 [==============================] - 10s 23ms/step - loss: 0.1416 - val_loss: 0.1676
Epoch: 3/10
450/450 [==============================] - 10s 23ms/step - loss: 0.0859 - val_loss: 0.0421
Epoch: 4/10
450/450 [==============================] - 10s 23ms/step - loss: 0.0308 - val_loss: 0.0133
Epoch: 5/10
450/450 [==============================] - 10s 23ms/step - loss: 0.0163 - val_loss: 0.0071
Epoch: 6/10
450/450 [==============================] - 11s 23ms/step - loss: 0.0125 - val_loss: 0.0151
Epoch: 7/10
450/450 [==============================] - 10s 23ms/step - loss: 0.0097 - val_loss: 0.0182
Epoch: 8/10
450/450 [==============================] - 10s 23ms/step - loss: 0.0076 - val_loss: 0.0138
Epoch: 9/10
450/450 [==============================] - 10s 23ms/step - loss: 0.0065 - val_lo

# Adam

In [ ]:
torch.manual_seed(1)
np.random.seed(0)
Ws = initialize_weights()
step_size = 0.001
m0 = [torch.zeros(W.shape).to(device) for W in Ws]
v0 = [torch.zeros(W.shape).to(device) for W in Ws]
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
    save_start_condition(TrainLoss, TestLoss, times)
for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    trainacc = 0.0
    t0 = time.time()
    for (data, target) in train_loader:
        
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        trainloss += loss
        grads = grad(loss, Ws, create_graph=True)    

        with torch.no_grad():
            lmbd = min(n/(n+1), 0.9)
            m0 = [lmbd*old + (1.0-lmbd)*new for (old, new) in zip(m0, grads)]
            lmbd = min(n/(n+1), 0.999)
            v0 = [lmbd*old + (1.0-lmbd)*new*new for (old, new) in zip(v0, grads)]
            for (W,m,v) in zip(Ws, m0, v0):
                W -= step_size*(m/torch.sqrt(v + 1e-8))
            kbar.update(n, values=[("loss", loss.item())])    
            n = n + 1
            
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    
    
#     step_size = 0.01**(1/9)*step_size
    testloss = test_loss()
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
#     print('Epoch: {}; train loss: {}; test loss: {}, train_acc: {}, test_acc:{}, time: {}'\
#      .format(epoch+1, TrainLoss[-1], TestLoss[-1], TrainAcc[-1], TestAcc[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|████████████████████████████████████████| 450/450 [00:03<00:00, 123.82it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 12s 27ms/step - loss: 0.0819 - val_loss: 0.0047
Epoch: 2/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0103 - val_loss: 0.0015
Epoch: 3/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0020 - val_loss: 8.6111e-04
Epoch: 4/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0056 - val_loss: 4.6982e-04
Epoch: 5/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0035 - val_loss: 4.5371e-04
Epoch: 6/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0041 - val_loss: 3.3794e-04
Epoch: 7/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0011 - val_loss: 2.5227e-04
Epoch: 8/10
450/450 [==============================] - 13s 28ms/step - loss: 0.0018 - val_loss: 2.4613e-04
Epoch: 9/10
450/450 [==============================] - 13s 28ms/step

# PSGD

In [ ]:
torch.manual_seed(1)
np.random.seed(0)

Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
# Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []

times = []
save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
        kbar.update(n, values=[("loss", loss.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.01**(1/9)*step_size
    

scipy.io.savemat(results_dir + 'Kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 86.37it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 35s 79ms/step - loss: 0.0611 - val_loss: 0.0020
Epoch: 2/10
450/450 [==============================] - 35s 77ms/step - loss: 0.0013 - val_loss: 9.2140e-04
Epoch: 3/10
450/450 [==============================] - 35s 77ms/step - loss: 6.9965e-04 - val_loss: 5.7163e-04
Epoch: 4/10
450/450 [==============================] - 34s 76ms/step - loss: 4.7069e-04 - val_loss: 4.1109e-04
Epoch: 5/10
450/450 [==============================] - 35s 77ms/step - loss: 3.5105e-04 - val_loss: 3.1323e-04
Epoch: 6/10
450/450 [==============================] - 34s 77ms/step - loss: 2.7345e-04 - val_loss: 2.4660e-04
Epoch: 7/10
450/450 [==============================] - 34s 77ms/step - loss: 2.1767e-04 - val_loss: 1.9946e-04
Epoch: 8/10
450/450 [==============================] - 34s 76ms/step - loss: 1.7911e-04 - val_loss: 1.6705e-04
Epoch: 9/10
450/450 [===================

# DPSGD APPROACH 1

In [ ]:
_tiny = 1.2e-38 
 # pi = (torch.trace(Ql)*Qr.shape[0])/(torch.trace(Qr)*Ql.shape[0])
    # 

    
def precond_grad_kron(Ql, Qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd))*IR)

    return P1.mm(Grad).mm(P2)

def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    # print(r, M, lambd)
    if r > 3/4:
      lambd = lambd*omega
    elif r < 1/4:
      lambd = lambd / omega
    return lambd


In [ ]:
torch.manual_seed(1)
np.random.seed(0)

Ws = initialize_weights()
# Qs = [[0.1*torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]

Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after

eta = 1e-10
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(-step_size * pg*psgd.precond_grad_kron(q[0], q[1], -step_size * pg)) \
                #              for (g, pg, q) in zip(grads, pre_grads, Qs)])
                loss2 = F.mse_loss(model(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)

        kbar.update(n, values=[("loss", loss.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)

    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.01**(1/9)*step_size
    

scipy.io.savemat(results_dir + 'Kron_damped.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 87.37it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 35s 79ms/step - loss: 0.0576 - val_loss: 0.0024
Epoch: 2/10
450/450 [==============================] - 35s 78ms/step - loss: 0.0015 - val_loss: 9.9874e-04
Epoch: 3/10
450/450 [==============================] - 35s 77ms/step - loss: 7.6693e-04 - val_loss: 6.2921e-04
Epoch: 4/10
450/450 [==============================] - 35s 77ms/step - loss: 5.3785e-04 - val_loss: 4.7327e-04
Epoch: 5/10
450/450 [==============================] - 35s 78ms/step - loss: 4.3112e-04 - val_loss: 3.9303e-04
Epoch: 6/10
450/450 [==============================] - 35s 78ms/step - loss: 3.6721e-04 - val_loss: 3.4055e-04
Epoch: 7/10
450/450 [==============================] - 35s 78ms/step - loss: 3.2185e-04 - val_loss: 3.0138e-04
Epoch: 8/10
450/450 [==============================] - 35s 78ms/step - loss: 2.8482e-04 - val_loss: 2.6462e-04
Epoch: 9/10
450/450 [===================

# DPSGD APPROACH 2

In [ ]:
torch.manual_seed(1)
np.random.seed(0)

Ws = initialize_weights()
# Qs = [[0.1*torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]

Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
dQs = [[torch.ones(W.shape[0],1).to(device), torch.ones(1,W.shape[1]).to(device)] for W in Ws]

step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
# 
eta = 1e-10
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            damp_grads = [((lambd+eta)**0.5)*g for g in grads]
            pre_grads = [pg+dg for (pg, dg) in zip(pre_grads, damp_grads)] 
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(-step_size * pg*psgd.precond_grad_kron(q[0], q[1], -step_size * pg)) \
                #              for (g, pg, q) in zip(grads, pre_grads, Qs)])
                loss2 = F.mse_loss(model(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)

        kbar.update(n, values=[("loss", loss.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)

    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.01**(1/9)*step_size
    

scipy.io.savemat(results_dir + 'Kron_damped2.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 86.60it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 35s 77ms/step - loss: 0.0675 - val_loss: 0.0022
Epoch: 2/10
450/450 [==============================] - 34s 76ms/step - loss: 0.0014 - val_loss: 9.9354e-04
Epoch: 3/10
450/450 [==============================] - 34s 75ms/step - loss: 8.0202e-04 - val_loss: 6.3238e-04
Epoch: 4/10
450/450 [==============================] - 34s 76ms/step - loss: 5.3391e-04 - val_loss: 4.3072e-04
Epoch: 5/10
450/450 [==============================] - 34s 76ms/step - loss: 3.7209e-04 - val_loss: 3.0525e-04
Epoch: 6/10
450/450 [==============================] - 34s 76ms/step - loss: 2.6700e-04 - val_loss: 2.2828e-04
Epoch: 7/10
450/450 [==============================] - 35s 77ms/step - loss: 2.0267e-04 - val_loss: 1.8017e-04
Epoch: 8/10
450/450 [==============================] - 34s 76ms/step - loss: 1.6261e-04 - val_loss: 1.4853e-04
Epoch: 9/10
450/450 [===================

# DPSGD-M

In [ ]:
def precond_kron(Ql, Qr, Pl, Pr, beta):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd))*IR)

    Pl = beta*Pl + (1-beta)*P1 
    Pr = beta*Pr + (1-beta)*P2 

    return [P1, P2, Pl, Pr]

def precond_kron2(Ql, Qr, Pl, Pr, beta):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    Pl = beta*Pl + (1-beta)*P1 
    Pr = beta*Pr + (1-beta)*P2 
    return [P1, P2, Pl, Pr]

def precond_grad_kron2(Pl, Pr, Grad):
    return Pl.mm(Grad).mm(Pr)

torch.manual_seed(1)
np.random.seed(0)

Ws = initialize_weights()
# Qs = [[0.1*torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]

Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
Ps = [[torch.zeros(W.shape[0]).to(device), torch.zeros(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-10
beta = 0.7

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    n = 0
    trainloss = 0.0
    t0 = time.time()
   
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss= train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            beta = min(n/(n+1), 0.7)
            Ps = [precond_kron(q[0], q[1], p[0], p[1], beta) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-10:
              
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(model(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M, lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)

    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.01**(1/9)*step_size
    

scipy.io.savemat(results_dir + 'mod_psgd1.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 88.57it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 36s 80ms/step - loss: 0.0601 - val_loss: 0.0024
Epoch: 2/10
450/450 [==============================] - 35s 78ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch: 3/10
450/450 [==============================] - 35s 78ms/step - loss: 7.8749e-04 - val_loss: 6.5568e-04
Epoch: 4/10
450/450 [==============================] - 35s 78ms/step - loss: 5.4210e-04 - val_loss: 4.9084e-04
Epoch: 5/10
450/450 [==============================] - 35s 78ms/step - loss: 4.2312e-04 - val_loss: 3.8847e-04
Epoch: 6/10
450/450 [==============================] - 35s 78ms/step - loss: 3.5095e-04 - val_loss: 3.2957e-04
Epoch: 7/10
450/450 [==============================] - 35s 78ms/step - loss: 3.0060e-04 - val_loss: 2.8617e-04
Epoch: 8/10
450/450 [==============================] - 35s 78ms/step - loss: 2.6151e-04 - val_loss: 2.4687e-04
Epoch: 9/10
450/450 [=======================

# dpsgd-m 2

In [ ]:

torch.manual_seed(1)
np.random.seed(0)

Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]

# Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
dQs = [[torch.ones(W.shape[0],1).to(device), torch.ones(1,W.shape[1]).to(device)] for W in Ws]

step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 10
omega = (19/20)**update_after
beta = 0.7
eta = 1e-10
for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)
        
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            beta = min(n/(n+1), 0.7)
            Ps = [precond_kron2(q[0], q[1], p[0], p[1], beta) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            
            damp_grads = [((lambd+eta)**0.5)*g for g in grads]
            pre_grads = [pg+dg for (pg, dg) in zip(pre_grads, damp_grads)] 
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)

            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(g*pg) for (g, pg) in zip(grads, pre_grads)])
                # M = 0.5*sum([torch.sum(-step_size * pg*psgd.precond_grad_kron(q[0], q[1], -step_size * pg)) \
                #              for (g, pg, q) in zip(grads, pre_grads, Qs)])
                loss2 = F.mse_loss(model(data), target)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)

        kbar.update(n, values=[("loss", loss.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)

    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.01**(1/9)*step_size
    

scipy.io.savemat(results_dir + 'mod_psgd2.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 87.00it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 34s 76ms/step - loss: 0.0654 - val_loss: 0.0027
Epoch: 2/10
450/450 [==============================] - 34s 76ms/step - loss: 0.0017 - val_loss: 0.0012
Epoch: 3/10
450/450 [==============================] - 34s 76ms/step - loss: 8.8396e-04 - val_loss: 6.5517e-04
Epoch: 4/10
450/450 [==============================] - 34s 76ms/step - loss: 5.2386e-04 - val_loss: 4.1191e-04
Epoch: 5/10
450/450 [==============================] - 34s 76ms/step - loss: 3.5349e-04 - val_loss: 2.8831e-04
Epoch: 6/10
450/450 [==============================] - 34s 76ms/step - loss: 2.5486e-04 - val_loss: 2.1568e-04
Epoch: 7/10
450/450 [==============================] - 34s 76ms/step - loss: 1.9678e-04 - val_loss: 1.7175e-04
Epoch: 8/10
450/450 [==============================] - 35s 77ms/step - loss: 1.5988e-04 - val_loss: 1.4320e-04
Epoch: 9/10
450/450 [=======================

# Shampoo

In [ ]:
torch.manual_seed(1)
np.random.seed(0)

Ws = initialize_weights()
Qs = [[0.01*torch.eye(W.shape[0]).to(device), 0.01*torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

def matrix_power(matrix, power):
    # use CPU for svd for speed up
    matrix = matrix.cpu()
    u, s, v = torch.svd(matrix)
    return (u @ s.pow_(power).diag() @ v.t()).cuda()

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        
      
        data, target = data.to(device), target.to(device)
        loss = train_loss(data, target)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        with torch.no_grad():
            Qs = [[q[0] + g.mm(g.t()), q[1] + (g.t()).mm(g)] for (q, g) in zip(Qs, grads)]
            inv_Qs = [[matrix_power(q[0], -1/4), matrix_power(q[1], -1/4)]for q in Qs]
            pre_grads = [q[0].mm(g).mm(q[1]) for (q, g) in zip(inv_Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

        kbar.update(n, values=[("loss", loss.item())])
        n += 1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.1**(1/9)*step_size
   

scipy.io.savemat(results_dir + 'shampoo.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

  0%|                                                   | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 89.22it/s]


Epoch: 0; train loss: 0.20708046807183159; test loss: 0.20636802673339844,  time: 0
Epoch: 1/10
450/450 [==============================] - 14s 31ms/step - loss: 0.0587 - val_loss: 0.0049
Epoch: 2/10
450/450 [==============================] - 14s 31ms/step - loss: 0.0028 - val_loss: 0.0019
Epoch: 3/10
450/450 [==============================] - 14s 32ms/step - loss: 0.0016 - val_loss: 0.0013
Epoch: 4/10
450/450 [==============================] - 14s 31ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch: 5/10
450/450 [==============================] - 14s 31ms/step - loss: 8.5416e-04 - val_loss: 7.2847e-04
Epoch: 6/10
450/450 [==============================] - 14s 31ms/step - loss: 7.0286e-04 - val_loss: 5.9225e-04
Epoch: 7/10
450/450 [==============================] - 14s 31ms/step - loss: 5.3682e-04 - val_loss: 3.5852e-04
Epoch: 8/10
450/450 [==============================] - 14s 31ms/step - loss: 5.1949e-04 - val_loss: 6.3872e-04
Epoch: 9/10
450/450 [==============================] - 14s 3

# Comparison

In [ ]:
# opts = ['adam','Kron','SCAN','SCAW', 'fisher_kron','fisher_SCAN','fisher_SCAW','Kron_test','Kron_test2','Kron_damped','fisher_kron_damped','KFAC', 'shampoo']
opts = ['SGD','adam','Kron','Kron_damped2','mod_psgd1', 'shampoo']

total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	

In [ ]:
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF', '#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']
# colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [ ]:
for opt in opts:
    # print(opt)
    data = opts_data[opt]
    times[opt] = data.get('Time')
    train_times[opt] = np.cumsum(times[opt])
    test_times[opt] = np.cumsum(times[opt])
    total_train_time[opt] = np.sum(times[opt])
    train_losses[opt] = data.get('TrainLoss').reshape(EPOCHS+1,)
    test_losses[opt] = data.get('TestLoss').reshape(EPOCHS+1,)


In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')